In [ ]:
%pip install openai langchain tiktoken wikipedia langchain-experimental lanchainhub docarray -q

# Aplication Development

## Repaso rapido

### Prompts


Los "prompts" son instrucciones o frases cortas que se utilizan para guiar a un modelo de lenguaje como un Modelo de Lenguaje de Aprendizaje de Máquina (LLM) sobre qué tipo de texto generar. Estas instrucciones pueden ser tan simples como una palabra o frase, o pueden ser párrafos completos, dependiendo del tipo de respuesta deseada y del modelo específico que se esté utilizando.

### `Langchain`

`LangChain` is a *framework* for developing applications powered by language models. It enables applications that:

Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)
Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)

<!-- 
### Langchain: Question and Answer with DocArrayInMemorySearch

Anteriormente se uso Chroma para almacenar la base de datos, ahora se usara `DocArrayInMemorySearch` para hacer las consultas, el procedimiento es similar -->


<!-- #### 

- Question and Answer
  - leer un documento
  - guardar el documento en una base de datos pasando previamente por un modelo de embeddings
  - hacer las consultas
  - la base de datos retorna una lista de elementos que segun la coincidencia de vectoores, son importantes para responder la consulta
  - los documentos se pasan por un motor de llm para sintetizar y generar una respuesta adecuada  -->


## Model Parsel

### Chat API: Open AI

- [Open ai version 1.0.0](https://github.com/openai/openai-python/discussions/742)

En internet se puede encontrar varios tutoriales de como usar la API de openai, pero 
si estos tutoriales son antes de noviembre es posible que esten desactualizados, ya que si se instala `pip install openai` se tendra una version de 1.x.x, mientras que los tutoriales antes de esa fecha estaban trabajando con la version beta.

Por ejemplo para poder acceder al chat de `openai`, note en el siguiente ejemplo que el nuevo modelo de creacion se basa en crear una instancia del modulo `OpenAI()` en vez de tenerlo globalmente, y para acceder a los metodos pasamos de `openai.ChatCompletition` a `OpenAI().chat.completions`

```python
# old
import openai

completion = openai.ChatCompletion.acreate(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])

# new
from openai import OpenAI()

client = OpenAI()
completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
```

En cuanto a las respuestas, se basa en [`pydantic models`](https://docs.pydantic.dev/latest/concepts/models/) lo que quiere decir que ya no se trabajan con diccionarios sino mas bien con atributos, sin embargo `pydantic` permite convertir a un diccionario con `model.model_dump()`.

```python

# old 
import json
import openai

completion = openai.Completion.create(model='gpt-3.5-turbo')
print(completion['choices'][0]['text']) # /// old
print(completion.get('usage')) #  /// old
print(json.dumps(completion, indent=2))

# new
from openai import OpenAI

client = OpenAI()

completion = client.completions.create(model='gpt-3.5-turbo')
print(completion.choices[0].text)  #/// new
print(dict(completion).get('usage'))    #/// new
print(completion.model_dump_json(indent=2))

```


*Example*

Primero debemos de llamar a la clase,  crear el cliente de OpenAI, y pasar como 
parametro nuestra api_key sin embargo si se esta usando archivos de entorno `.env` solo se debe tener correctamente el nombre de la variable en este caso `OPENAI_API_KEY`. En el cual podemos definir nuestra api_key

![](https://imgs.search.brave.com/j6GBXNcpKlQxBOvfNFyojLGZjzHPUAYi71MoT5kmFes/rs:fit:860:0:0/g:ce/aHR0cHM6Ly9sYWJz/LnRoaW5rdGVjdHVy/ZS5jb20vc3RvcmFn/ZS9pbWFnZS01LnBu/Zw)

Para poder acceder a la variable debemos de correr el siguiente codigo y llamar a nuestra variable de entorno

In [ ]:
from google.colab import userdata
import os
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

Para poder acceder directamente a nuestra respuesta, se crea una funcion la cual toma como parametros la pregunta (o prompt) y el modelo que queremos usar, recalcando como ahora se usa respuestas de pydantic entonces debemos acceder a la respuesta con los atributos en este caso es `completion.choices[0].message.content`. Problemos con una pregunta general y un prompt.

*Example*


### Open AI and LangChain

LangChain afortunadamente actualiza su codigo conforme sus dependencias tambien
lo hacen, por lo que en las ultimas versiones de langchain ya esta implementada esta migracion.

Para poder usar esta forma de llamar a openai, podemos replicar el anterior ejemplo con prompts, primero llamamos al modulo de `ChatOpenAI`, y definimos el prompt con las variables que queremos, y lo guardamos en la variable message.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat # callable([list])

template_string = """"""
prompt_template = ChatPromptTemplate.from_template(template_string)
variable1
variable2

message = prompt_template.format_messages(
    # 
)

Por ultimo obtenemos la respuesta.

In [ ]:
response = chat(message)
print(response.content)

## Output Parser

En algunos casos se querra que el tipo de respuesta tenga alguna estructura, por ejemplo un json, sin embargo cuando obtenemos la respuesta de langchain notamos que el resultado es un String y no podemos acceder a los elementos. 


Afortunadamente se puede pasar el output string a un dicionario de python, primero importamos `ResponseSchema` y `StructuredOutputParser`, luego creamos los elemetos que se quiere extraer con una descripcion para que el modelo sepa que colocar y parsear al typo requerido, por ultimo pasamos esto por las instrucciones de formato, dentro del prompt inicial.


<!-- codigo -->

Cuando se corra el modelo se obtiene una respuesta tal cual se puso en las instrucciones de formato pasamos por el metodo parse del contenido y este retornara el diccionario de python.

<!-- Adicionalmente se puede consultar https://www.youtube.com/watch?v=I4mFqyqFkxg -->

## LangChain Memory

usualmente cuando interactuamos con `ChatGPT`, las respuestas que tenemos de una a la siguiente estan relacionadas ya que `ChatGOT` trabaja contextualizando (chains) y tomando como referencia los inputs y outputs anteriores para formular la ultima respuesta.

![](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

Para poder simular este escenario con LangChain, usaremos 4 tipos de memoria y 

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

Para poder usar estos metodos podemos usar esta pequena referencia

In [ ]:
from langchain.chat_models import ChatOpenAI #///llm model
from langchain.chains import ConversationChains # chain

llm_model = 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0.0, model=llm_model)

```python
# ///// only reference
from langchain.memory import Conversation{Method}Memory as Memory

llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')
memory = Memory()
conversation = ConversationChains(
    llm = llm,
    memory = memory
)

conversation.predict(input='Question') # Add memory (input, output)
memory.load_memory_variables({}) # Returns memory (history)
memory.save_context({"input": "user_content", "output": "AI response"}) # Add manual memory
```

### ConversationBufferMemory

Using this in a chain (setting verbose=True so we can see the prompt).

In [ ]:
from langchain.memory import ConversationBufferMemory as Memory 

memory = Memory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="Tell me about yourself.")

In [ ]:
conversation.load_memory_variables({})

### ConversationBufferWindowMemory

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferMemory as Memory 

memory = Memory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

### ConversationTokenBufferMemory

`ConversationTokenBufferMemory` keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

In [ ]:
from langchain.memory import ConversationBufferMemory as Memory 

memory = Memory(max_token_limit = 60)

conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

conversation_with_summary.predict(input="Hi, what's up?")
conversation_with_summary.predict(input="Just working on writing some code!")

### ConversationSummaryMemory

`ConversationSummaryBufferMemory` combines the two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. It uses token length rather than number of interactions to determine when to flush interactions.

In [ ]:
from langchain.memory import ConversationBufferMemory as Memory

schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = Memory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

conversation.predict(input="What would be a good demo to show?")
memory.load_memory_variables({})

## LangChain: Chains

Anteriormente se uso cadenas, pero de manera rapida para poder obtener una respuesta 
del modelo llm.

Chains in LangChain encompass sequences of function calls, whether to an LLM, a tool, or a data preprocessing step. LCEL (LangChain Execution Language) is the primary method for constructing these chains, offering both custom and off-the-shelf options. 



Combine multiple chains where the output of the one chain is the input of the next chain

- LLMchain:  It's just the combination of the LLM and the prompt (simple chain)

- simple sequatial chain:  un cadena tiene un output el cual es el input de la cadena 2 el cual tiene el output final

![](https://miro.medium.com/v2/resize:fit:1400/1*sYdb7ca9vcmDV0gOiNaruQ.png)

- sequential chain: Comparing it to the above chain, you can notice that any 
step in the chain can take in multiple input variables. 
This is useful when you have more complicated downstream 
chains that need to be a composition of multiple 
previous chains. 

![](https://av-eks-lekhak.s3.amazonaws.com/media/__sized__/article_images/7_Bpn6EWD-thumbnail_webp-600x300.webp)

- Router chain: This chain uses an LLM to route between potential options. If you have multiple sub chains, 
each of which specialized for a particular type of input, 
you could have a router chain which first 
decides which subchain to pass it to and then passes it to 
that chain. 

![](https://miro.medium.com/v2/resize:fit:700/1*0TDSAfaL2Q46TnFWkQTagg.jpeg)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate as CPrompt
from langchain.chains import LLMChain # simple chain
from langchain.chains import SimpleSequentialChain # Simple Sequential Chain
from langchain.chains import SequentialChain # Sequential Chain
# Router Chain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

llm_model = 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0.9, model=llm_model)

### LLM chain:

In [ ]:
prompt = CPrompt.from_template("{input_}")
chain = LLMChain(llm = llm, prompt = prompt)
input_ = ""
chain.run(input_)

### SimpleSequentialChain

In [ ]:
first_prompt = CPrompt.from_template("{input_1}")
first_chain = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = CPrompt.from_template("{input_2}")
second_chain = LLMChain(llm=llm, prompt=second_prompt)

simple_chain = SimpleSequentialChain(
    chains = [first_chain, second_chain],
    verbose = True
)

input_1 = ""

simple_chain.run(input_1)

### SequentialChain

chain one

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

Chain Two

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

Chain Three

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

Chain Four

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

Run with review input

In [ ]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

overall_chain(review)

### Router Chain

Cotextual Inputs and Prompts

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

Make the prompts destination templates

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Template for multi prompt router template


In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
    ```json
    {{{{
        "destination": string \ name of the prompt to use or "DEFAULT"
        "next_inputs": string \ a potentially modified version of the original input
    }}}}
    ```\

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

and finally create the final router prompt, router_chain 

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

Create the llm interpretter

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain, 
    verbose=True # False if you don't want to see the procces of the chain
)

*Test our interpreter*

In [ ]:
# Physic Interpreter
chain.run("What is black body radiation?")

In [ ]:
# Math interpreter
chain.run("what is square of (16)")

Call the default chain which itself is just a LLMChain

In [ ]:
# Not related / Biology
chain.run("Why does every cell in ourt body contain DNA?")

For more details consult [langchain Chains API](https://python.langchain.com/docs/modules/chains)

## Agents

Usualmente los modelos de lenguaje natural, se basan en datos preprocesados hasta un determinado tiemp, tomando en cuenga a GPT-3.5 el cual tiene informacion hasta el anio 2022. Sin embargo la creacion de conocimiento continua o el prompt necesita de herramientas especificas, para poder utilizar esta informacion y sintetisarlo podemos usar los conocimientos previos los cuales en escencia es, tener vvarias piezas de informacion y poder crear contenido que sea relevante, la diferencia es que ahora vamos a consultar a informacion de la web y otras herramientas para poder recopilar informacion y sintetizar la informacion para responder a la consulta. 


### Wikipedia

En especifico, cuando se hace alguna consulta al modelo de llm, y no encuentra en el endpoint, los "`tools`" buscara informacion en otras fuentes de informacion que especifiquemos para poder generar el contexto y respuesta a la pregunta. Para ello vamos a usar los componentes `tools` de langchain, en especifico para simular el caso hipotetico informacion que se encuentra en wikypedia, se puede consultar mas tools en [`Components.tools`](https://python.langchain.com/docs/integrations/tools)

In [ ]:
# Methods and Fucntions
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
llm_model = 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0, model=llm_model)
# add tools into a list
tools = load_tools(["wikipedia"], llm=llm)
# initialize agent
wikipedia = initialize_agent(
    tools,  #wiki
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True # show 
    )

*Example*

In [ ]:
wikipedia("who is the current president of Peru?")

In [ ]:
# Chile: Sebastian Pinera https://en.wikipedia.org/wiki/Sebasti%C3%A1n_Pi%C3%B1era
wikipedia("Sebastian Piñera die? cause of death")

### Python REPL

Este se usa para poder ejecutar python conforme a la pregunta que se le haga, para poder dar solucion a la respuesta

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_openai_functions_agent
# tools = [PythonREPLTool()]

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

tools = [PythonREPLTool()]

agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

*Example*


In [ ]:
agent_executor.invoke({"input": "What is the 10th fibonacci number?"})

In [ ]:
customer_list = [
    ["Harrison", "Chase"],
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"],
    ["Geoff", "Fusion"],
    ["Trance", "Former"],
    ["Jen", "Ayai"],
]
agent_executor.invoke(
    {
        "input": f"""
        
        
        Sort these customers by \
        last name and then first name \
        and print the output: {customer_list}
        """
    }
)